In [12]:
# List of users, a simple list of dict with username, role and password as key

users = [
    {"username": "admin", "role": "administrator", "password": "admin"},
    {"username": "alice", "role": "user", "password": "alice"},
    {"username": "bob", "role": "user", "password": "bob"}
]
# List of documents, again a list of dict with id of the document, title, content and owner
documents = [
    {"id": 1, "title": "Company Strategy", "content": "Secret business plans...", "owner": "admin", "visibility": "private"},
    {"id": 2, "title": "Project Report", "content": "Q1 project results...", "owner": "alice", "visibility": "private"},
    {"id": 3, "title": "Personal Notes", "content": "My private thoughts...", "owner": "alice", "visibility": "private"},
    {"id": 4, "title": "Public Info", "content": "Company public information...", "owner": "admin", "visibility": "public"}
]

def user_login(username, prompted_password):
    """
    Authenticate a user by username and password.

    Parameters
    ----------
    username : str
        The username to authenticate
    prompted_password : str
        The password provided by the user during login attempt

    Returns
    -------
    user_dict : dict or None
        User dictionary if username exists, None otherwise
    authenticated : bool
        True if authentication successful, False otherwise

    Notes
    -----
    Return scenarios:
    - (user_dict, True): Successful authentication
    - (user_dict, False): User exists but wrong password  
    - (None, False): User doesn't exist
    """
    for user in users:
        if user['username'] == username:
            if user['password'] == prompted_password:
                return user, True
            else:
                return user, False
    return None, False

def get_document_index(document_title):
    """
    Retrieve the document index by title if exists.

    Parameters
    ----------
    document_title : str
        The exact title of the document to retrieve

    Returns
    -------
    index: int or None
        Document index in the list, None otherwise
    """
    for index, doc in enumerate(documents):
        if doc['title'] == document_title:
            return index
    else:
        return None
    



def retrive_document(document_title, connected_user):
    """
    Retrieve a document by title if the user has permission to view it.

    Parameters
    ----------
    document_title : str
        The exact title of the document to retrieve
    connected_user : dict
        User dictionary containing 'role' and 'username' keys

    Returns
    -------
    document : dict or None
        Document dictionary if found and user has permission, None otherwise

    Notes
    -----
    Permission hierarchy:
    1. Administrators: access to all documents
    2. Document owners: access to their own documents regardless of visibility
    3. All users: access to public documents
    4. Otherwise: access denied
    """
    doc_index = get_document_index(document_title)
    if doc_index is None:
        return None
    doc = documents[doc_index]

    # check if user is authorized to see the document before returning it
    if connected_user['role'] == 'administrator' \
            or doc['visibility'] == 'public' \
            or doc['owner'] == connected_user['username']:
        return doc
    return None

def change_document_visibility(document_title, connected_user, new_visibility):
    """
    Change the visibility of a document if the user has permission.

    Parameters
    ----------
    document_title : str
        The exact title of the document to modify
    connected_user : dict
        User dictionary containing 'role' and 'username' keys
    new_visibility : str
        The new visibility setting for the document

    Returns
    -------
    success : bool
        True if visibility was successfully changed, False otherwise

    Notes
    -----
    Permission rules:
    - Document owners can always change visibility of their documents
    - Administrators can change visibility of any document
    - Other users cannot change document visibility
    """
    doc_index = get_document_index(document_title)
    if doc_index is None:
        return False
    doc = documents[doc_index]

    # check if user is authorized to see the document before returning it
    has_right = False
    if connected_user['role'] == 'administrator' \
            or doc['owner'] == connected_user['username']:
        has_right = True

    if not has_right:
        return False
        
    documents[doc_index]['visibility'] = new_visibility
    return True




In [13]:
def print_test_result(test_name, expected, actual, success_condition=True):
    """Helper function to print test results"""
    status = "PASS" if actual == expected else "FAIL"
    if not success_condition:
        status = "PASS" if actual == expected else "FAIL"

    print(f"  {test_name}: {status}")
    print(f"    Expected: {expected}")
    print(f"    Actual: {actual}")
    if actual != expected:
        print(f"    ⚠️  MISMATCH!")
    print()

def print_section(title):
    """Print section headers"""
    print("\n" + "=" * 50)
    print(title)
    print("=" * 50)

# ============================================================================
# TEST 1: USER LOGIN FUNCTIONALITY
# ============================================================================

print_section("TEST 1: USER LOGIN FUNCTIONALITY")

print("1.1 Testing successful login:")
user, status = user_login("alice", "alice")
print_test_result("Alice login with correct password", (users[1], True), (user, status))

print("1.2 Testing wrong password:")
user, status = user_login("alice", "wrongpassword")
print_test_result("Alice login with wrong password", (users[1], False), (user, status))

print("1.3 Testing non-existent user:")
user, status = user_login("charlie", "anypassword")
print_test_result("Non-existent user login", (None, False), (user, status))

print("1.4 Testing admin login:")
user, status = user_login("admin", "admin")
print_test_result("Admin login with correct password", (users[0], True), (user, status))

# ============================================================================
# TEST 2: DOCUMENT RETRIEVAL FUNCTIONALITY
# ============================================================================

print_section("TEST 2: DOCUMENT RETRIEVAL FUNCTIONALITY")

# First, let users log in
admin_user, _ = user_login("admin", "admin")
alice_user, _ = user_login("alice", "alice")
bob_user, _ = user_login("bob", "bob")

print("2.1 Admin accessing private document (should work):")
doc = retrive_document("Company Strategy", admin_user)
expected_doc = documents[0]
print_test_result("Admin accessing private doc", expected_doc, doc)

print("2.2 Owner accessing their own private document (should work):")
doc = retrive_document("Project Report", alice_user)
expected_doc = documents[1]
print_test_result("Owner accessing own private doc", expected_doc, doc)

print("2.3 Non-owner accessing private document (should fail):")
doc = retrive_document("Project Report", bob_user)  # Bob trying to access Alice's doc
print_test_result("Non-owner accessing private doc", None, doc)

print("2.4 Any user accessing public document (should work):")
doc = retrive_document("Public Info", bob_user)
expected_doc = documents[3]
print_test_result("Any user accessing public doc", expected_doc, doc)

print("2.5 Accessing non-existent document (should fail):")
doc = retrive_document("Non-Existent Document", admin_user)
print_test_result("Accessing non-existent doc", None, doc)

# ============================================================================
# TEST 3: DOCUMENT VISIBILITY CHANGE FUNCTIONALITY
# ============================================================================

print_section("TEST 3: DOCUMENT VISIBILITY CHANGE FUNCTIONALITY")

print("3.1 Admin changing any document visibility (should work):")
original_visibility = documents[1]['visibility']  # Save original
result = change_document_visibility("Project Report", admin_user, "public")
print_test_result("Admin changing document visibility", True, result)
print(f"    Document visibility changed from '{original_visibility}' to 'public'")

# Verify the change actually happened
doc = retrive_document("Project Report", bob_user)  # Bob should now see it
print_test_result("Bob can now see the changed document", documents[1], doc, doc is not None)

# Restore original visibility for further tests
documents[1]['visibility'] = original_visibility

print("3.2 Owner changing their own document visibility (should work):")
result = change_document_visibility("Project Report", alice_user, "public")
print_test_result("Owner changing own document visibility", True, result)

# Restore for further tests
documents[1]['visibility'] = original_visibility

print("3.3 Non-owner changing document visibility (should fail - BUG DEMONSTRATION):")
print("   Testing the bug: non-owner trying to change public document visibility")
result = change_document_visibility("Public Info", bob_user, "private")
print_test_result("Non-owner changing public doc (BUG)", False, result)
print("   ⚠️  This should be False, but let's check if the bug exists...")

# Let's demonstrate the bug more clearly
print("\n3.4 Demonstrating the permission logic bug:")
print("   Current permission check in change_document_visibility:")
print("   if connected_user['role'] == 'administrator' or doc['visibility'] == 'public':")
print("       has_right = True")
print("   This means ANY user can change visibility of PUBLIC documents!")
print("   Let's test this:")

# Make sure we have a public document
documents[3]['visibility'] = 'public'

# Bob (non-owner) tries to change a public document
result = change_document_visibility("Public Info", bob_user, "private")
if result:
    print("   BUG CONFIRMED: Bob (non-owner) was able to change public document!")
    print("   This happens because the condition checks for 'public' visibility")
    print("   without checking ownership!")
else:
    print("   Bug appears to be fixed")

# Restore public visibility
documents[3]['visibility'] = 'public'

print("3.5 Changing non-existent document visibility (should fail):")
result = change_document_visibility("Non-Existent Doc", admin_user, "public")
print_test_result("Changing non-existent doc visibility", False, result)

# ============================================================================
# TEST 4: INTEGRATION TESTS - REAL-WORLD SCENARIOS
# ============================================================================

print_section("TEST 4: REAL-WORLD SCENARIOS")

print("4.1 Complete workflow: Alice creates private doc, then makes it public")
# Simulate Alice creating a new private document
new_doc = {"id": 5, "title": "Alice's New Report", "content": "New content...", "owner": "alice", "visibility": "private"}
documents.append(new_doc)
print("   Created new private document for Alice")

# Bob tries to access it (should fail)
doc = retrive_document("Alice's New Report", bob_user)
print_test_result("Bob cannot access Alice's private doc", None, doc)

# Alice changes visibility to public
result = change_document_visibility("Alice's New Report", alice_user, "public")
print_test_result("Alice changes her doc to public", True, result)

# Bob tries again (should now work)
doc = retrive_document("Alice's New Report", bob_user)
print_test_result("Bob can now access the public doc", new_doc, doc)

print("4.2 Admin overriding user permissions:")
# Alice tries to change admin's document (should fail with proper logic)
result = change_document_visibility("Company Strategy", alice_user, "public")
print_test_result("Alice changing admin's document", False, result)

# Admin changes it successfully
result = change_document_visibility("Company Strategy", admin_user, "public")
print_test_result("Admin changing any document", True, result)

# ============================================================================
# TEST 5: EDGE CASES AND ERROR CONDITIONS
# ============================================================================

print_section("TEST 5: EDGE CASES")

print("5.1 Empty strings and None values:")
user, status = user_login("", "")
print_test_result("Empty username login", (None, False), (user, status))

user, status = user_login("alice", "")
print_test_result("Empty password login", (users[1], False), (user, status))

doc = retrive_document("", admin_user)
print_test_result("Empty document title", None, doc)

print("5.2 Case sensitivity:")
user, status = user_login("ALICE", "alice")  # Different case
print_test_result("Case sensitive username", (None, False), (user, status))

doc = retrive_document("company strategy", admin_user)  # Different case
print_test_result("Case sensitive document title", None, doc)

# ============================================================================
# SUMMARY AND BUG ANALYSIS
# ============================================================================

print_section("TEST SUMMARY AND BUG ANALYSIS")

print("IDENTIFIED ISSUES:")
print("1. Permission Bug in change_document_visibility:")
print("   - Current logic: if doc is public OR user is admin → has rights")
print("   - This allows ANY user to change visibility of PUBLIC documents")
print("   - Fix: Only check for ownership OR administrator role")
print()
print("2. Double Loop Inefficiency:")
print("   - change_document_visibility uses two loops (find + update)")
print("   - Can be optimized to single loop")
print()
print("3. Missing Input Validation:")
print("   - No validation for new_visibility parameter")
print("   - Could accept invalid visibility values")

print("=" * 70)
print("TESTING COMPLETE")
print("=" * 70)


TEST 1: USER LOGIN FUNCTIONALITY
1.1 Testing successful login:
  Alice login with correct password: PASS
    Expected: ({'username': 'alice', 'role': 'user', 'password': 'alice'}, True)
    Actual: ({'username': 'alice', 'role': 'user', 'password': 'alice'}, True)

1.2 Testing wrong password:
  Alice login with wrong password: PASS
    Expected: ({'username': 'alice', 'role': 'user', 'password': 'alice'}, False)
    Actual: ({'username': 'alice', 'role': 'user', 'password': 'alice'}, False)

1.3 Testing non-existent user:
  Non-existent user login: PASS
    Expected: (None, False)
    Actual: (None, False)

1.4 Testing admin login:
  Admin login with correct password: PASS
    Expected: ({'username': 'admin', 'role': 'administrator', 'password': 'admin'}, True)
    Actual: ({'username': 'admin', 'role': 'administrator', 'password': 'admin'}, True)


TEST 2: DOCUMENT RETRIEVAL FUNCTIONALITY
2.1 Admin accessing private document (should work):
  Admin accessing private doc: PASS
    Expe